In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
def read_data_from_csv(filename):
    df = pd.read_csv(filename, sep='\t',encoding='utf-16')
    return df 
df = read_data_from_csv('raw_data.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tên SP        884 non-null    object
 1   Nhà sản xuất  631 non-null    object
 2   Model         263 non-null    object
 3   Kết nối       359 non-null    object
 4   Kích thước    461 non-null    object
 5   Loại switch   576 non-null    object
 6   Giá(đ)        884 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 48.5+ KB


,Tên SP,Nhà sản xuất,Model,Kết nối,Kích thước,Loại switch,Giá(đ)
0,Bàn Phím Cơ Gaming không dây Asus ROG Strix Sc...,NaN,NaN,NaN,NaN,NaN,3899000
1,Bàn phím Gaming ASUS ROG STRIX SCOPE II NX USB...,NaN,NaN,NaN,NaN,NaN,2699000
2,Bàn phím Gaming ASUS ROG AZOTH/NXSW/US/PBT/WHT...,NaN,NaN,USB 2.0 (TypeC sang TypeA),326 x 136 x 40 mm,NaN,6099000
3,Bàn phím Gaming Asus ROG Azoth NX SM Black _ 9...,Asus,90MP031B-BKUA01 (NX Snow Switch)\n90MP031A-BKU...,USB 2.0 (TypeC to TypeA)\nBluetooth 5.1\nRF 2....,326 x 136 x 40 mm,Đang cập nhật,5999000
4,Bàn phím Gaming ASUS ROG AZOTH NX RED/WL/PBT/O...,NaN,NaN,USB 2.0 (TypeC sang TypeA),326 x 136 x 40 mm,NaN,5999000
5,Bàn phím cơ Asus ROG Falchion Blue switch 90MP...,Asus,ROG Falchion,USB có dây \nWireless 2.4GHz,305x101x38.5 mm,Blue,1599000
6,Bàn phím Gaming Asus ROG Strix Scope II NX SM ...,Asus,90MP036A-BKUA00 (Snow Switch)\n90MP036B-BKUA00...,USB 2.0 (Type-C to Type-A),436 x 129 x 37mm,Đang cập nhật,3199000
7,Bàn phím game Asus TUF K1 (USB),Asus,NaN,NaN,Fullsize,Tactile TUF switch,849000
8,Kit bàn phím cơ FLEsport Q75 Crystal Grey (Cas...,NaN,NaN,NaN,NaN,NaN,1249000
9,Bàn phím Lecoo KB101 đen (USB),Lenovo,NaN,NaN,NaN,NaN,129000


In [36]:
def check_missing_data(df):
    # Thống kê lại dữ liệu trống sau khi thực hiện thay thế
    missing_data = df.isnull().sum()
    print(missing_data)
print("Thông kê dữ liệu trống ban đầu:")
check_missing_data(df)

Thông kê dữ liệu trống ban đầu:
Tên SP            0
Nhà sản xuất    253
Model           621
Kết nối         525
Kích thước      423
Loại switch     308
Giá(đ)            0
dtype: int64


In [37]:
def replace_missing_manufacturer(df):
    # Thay đổi giá trị 'Đang cập nhật' thành khoảng trắng
    df['Nhà sản xuất'] = df['Nhà sản xuất'].replace('Đang cập nhật', '')

    # Tạo một danh sách chứa tất cả các nhà sản xuất đã xuất hiện trong cột "Nhà sản xuất"
    manufacturers = df['Nhà sản xuất'].unique().tolist()
    
    # Duyệt qua từng dòng trong dataframe
    for index, row in df.iterrows():
        product_name = row['Tên SP']
        if isinstance(product_name, str):  # Kiểm tra xem giá trị của product_name có phải là chuỗi không
            # Duyệt qua từng nhà sản xuất đã từng xuất hiện
            for manufacturer in manufacturers:
                if isinstance(manufacturer, str):  # Kiểm tra xem giá trị của manufacturer có phải là chuỗi không
                    # Nếu tên nhà sản xuất có trong tên sản phẩm thì thay thế dữ liệu trống trong cột "Nhà sản xuất"
                    if manufacturer.lower() in product_name.lower():
                        df.at[index, 'Nhà sản xuất'] = manufacturer
                        break  # Sau khi thay thế xong, thoát khỏi vòng lặp
    
    return df

def replace_missing_model(df):
    # Thay đổi giá trị 'Đang cập nhật' thành khoảng trắng
    df['Model'] = df['Model'].replace('Đang cập nhật', '')

    # Tạo một danh sách chứa tất cả các model đã xuất hiện trong cột "Model"
    models = df['Model'].unique().tolist()
    
    # Duyệt qua từng dòng trong dataframe
    for index, row in df.iterrows():
        product_name = row['Tên SP']
        if isinstance(product_name, str):  # Kiểm tra xem giá trị của product_name có phải là chuỗi không
            # Duyệt qua từng model đã từng xuất hiện
            for model in models:
                if isinstance(model, str):  # Kiểm tra xem giá trị của model có phải là chuỗi không
                    # Nếu tên model có trong tên sản phẩm thì thay thế dữ liệu trống trong cột "Model"
                    if model.lower() in product_name.lower():
                        df.at[index, 'Model'] = model
                        break  # Sau khi thay thế xong, thoát khỏi vòng lặp
    
    return df
df = replace_missing_manufacturer(df)
df = replace_missing_model(df)
check_missing_data(df)

Tên SP            0
Nhà sản xuất     29
Model           581
Kết nối         525
Kích thước      423
Loại switch     308
Giá(đ)            0
dtype: int64


In [38]:
def check_and_remove_empty_rows(df):
    df['empty_count'] = df.isnull().sum(axis=1)
    df = df[df['empty_count'] < 3]
    df = df.drop(columns=['empty_count'])
    return df
df = check_and_remove_empty_rows(df)
check_missing_data(df)
df.info()

Tên SP            0
Nhà sản xuất      4
Model           319
Kết nối         231
Kích thước      144
Loại switch      71
Giá(đ)            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 582 entries, 2 to 878
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tên SP        582 non-null    object
 1   Nhà sản xuất  578 non-null    object
 2   Model         263 non-null    object
 3   Kết nối       351 non-null    object
 4   Kích thước    438 non-null    object
 5   Loại switch   511 non-null    object
 6   Giá(đ)        582 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 36.4+ KB


In [39]:
def replace_missing_value(df):
    # Thay đổi giá trị NaN thành 0 trong tất cả các cột
    df.fillna(0, inplace=True)
    
    # Thay đổi giá trị rỗng thành 0 trong tất cả các cột (nếu có)
    df.replace('', 0, inplace=True)
    df.replace('Đang cập nhật', 0, inplace=True)
    df.replace('nan', 0, inplace=True)
    
    return df
def remove_special_characters(text):
    # Kiểm tra nếu text không phải là chuỗi thì trả về text nguyên thể
    if isinstance(text, str):
        # Xóa các kí tự như "/", "(", "_" và toàn bộ kí tự phía sau
        text = re.sub(r'[/(_\n].*', '', text)
    return text
# bỏ chữ mm và các kí tự dư khác
def remove_special_charactersKichThuoc(text):
    # Kiểm tra nếu text không phải là chuỗi thì trả về text nguyên thể
    if isinstance(text, str):
        # Xóa các kí tự như "/", "(", "mm" và toàn bộ kí tự phía sau
        text = re.sub(r'[/(mm*xX×,.-].*', '', text)
    return text

def remove_commas(price):
    if isinstance(price, str):
        return price.replace(',', '')
    return price
def remove_rows_with_size_info(df):
    # Chuyển đổi cột "Kích thước" thành chuỗi
    df['Kích thước'] = df['Kích thước'].astype(str)

    # Tạo một danh sách các chuỗi mô tả kích thước
    size_info_strings = ["Độ dài", "Chiều rộng", "Chiều cao"]

    # Tạo một mặt nạ (mask) để xác định các hàng cần xóa
    mask = df['Kích thước'].str.contains('|'.join(size_info_strings), case=False)

    # Xóa các hàng dữ liệu theo mặt nạ
    df = df[~mask]

    return df

def add_zero_to_two_digits(value):
    if isinstance(value, str) and len(value) == 2:
        return value + '0'
    return value
def change_connection_values(df):
    df['Kết nối'] = df['Kết nối'].replace(['Dây', 'Có dây ', 'Có dây'], 'Có dây')
    return df
def update_connection_type(df):
    for index, row in df.iterrows():
        connection_type = row['Kết nối']
        if isinstance(connection_type, str):
            if 'Type C' in connection_type:
                df.at[index, 'Kết nối'] = 'vừa có dây vừa không dây'
            elif 'USB 2.0' in connection_type or connection_type == 'có dây' or connection_type == 'Có dây'or 'cáp liền' in connection_type:
                df.at[index, 'Kết nối'] = 'có dây'
            elif connection_type == '0':
                pass
            else:
                df.at[index, 'Kết nối'] = 'không dây'
    return df
def simplify_decimal_to_integer(text):
    # Kiểm tra nếu text không phải là chuỗi thì trả về text nguyên thể
    if isinstance(text, str):
        # Tìm kiếm số thập phân trong chuỗi và chuyển về số tự nhiên
        match = re.search(r'\d+\.\d+', text)
        if match:
            decimal_number = float(match.group())
            integer_number = int(decimal_number)
            text = re.sub(r'\d+\.\d+', str(integer_number), text)   
    return text
df['Tên SP'] = df['Tên SP'].apply(remove_special_characters)
df['Model'] = df['Model'].apply(remove_special_characters)
df['Kết nối'] = df['Kết nối'].apply(remove_special_characters)
df['Loại switch'] = df['Loại switch'].apply(remove_special_characters)
df['Giá(đ)'] = df['Giá(đ)'].apply(remove_commas)
df['Kích thước'] = df['Kích thước'].apply(remove_special_charactersKichThuoc)
df = remove_rows_with_size_info(df)
df['Kích thước'] = df['Kích thước'].apply(simplify_decimal_to_integer)
df['Kích thước'] = df['Kích thước'].apply(add_zero_to_two_digits)
df = replace_missing_value(df)
df = change_connection_values(df)
df = update_connection_type(df)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 575 entries, 2 to 878
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tên SP        575 non-null    object
 1   Nhà sản xuất  575 non-null    object
 2   Model         575 non-null    object
 3   Kết nối       575 non-null    object
 4   Kích thước    575 non-null    object
 5   Loại switch   575 non-null    object
 6   Giá(đ)        575 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 52.1+ KB


,Tên SP,Nhà sản xuất,Model,Kết nối,Kích thước,Loại switch,Giá(đ)
2,Bàn phím Gaming ASUS ROG AZOTH,Asus,0,có dây,326,0,6099000
3,Bàn phím Gaming Asus ROG Azoth NX SM Black,Asus,90MP031B-BKUA01,có dây,326,0,5999000
4,Bàn phím Gaming ASUS ROG AZOTH NX RED,Asus,0,có dây,326,0,5999000
5,Bàn phím cơ Asus ROG Falchion Blue switch 90MP...,Asus,ROG Falchion,không dây,305,Blue,1599000
6,Bàn phím Gaming Asus ROG Strix Scope II NX SM,Asus,90MP036A-BKUA00,có dây,436,0,3199000


In [40]:
def check_missing_data(df):
    # Thống kê lại dữ liệu trống sau khi thực hiện thay thế
    missing_data = df.isnull().sum()
    print(missing_data)
print("Thông kê dữ liệu trống ban đầu:")
check_missing_data(df)

Thông kê dữ liệu trống ban đầu:
Tên SP          0
Nhà sản xuất    0
Model           0
Kết nối         0
Kích thước      0
Loại switch     0
Giá(đ)          0
dtype: int64
